# WEED Inference
This notebook demonstrates how we integrate Earth Observation (EO) data processing with ONNX-based machine learning (ML) inference within the WEED framework.

Overview of the Process
1) Loading Data Cube:
We start by loading a data cube that contains all the enabled training features as individual bands. This lazy loading approach ensures efficient memory usage during data processing.

2) Model Metadata Retrieval:
The models are stored in an openEO-accessible storage site. Each model contains metadata specifying the features it was trained on.

Important Note: Users must add this metadata to the stored models.
The onnx_model_utilities module provides a utility function to extract metadata from a JSON file and embed it into the ONNX model. This approach is temporary and will be replaced as model training becomes fully integrated within the WEED framework.

In [1]:
import os
import sys
import openeo

from pathlib import Path
from eo_processing.utils.helper import init_connection, getUDFpath
from eo_processing.utils.onnx_model_utilities import get_name_output_cube_features

from eo_processing.openeo.processing import generate_master_feature_cube
from eo_processing.config import get_collection_options,  get_standard_processing_options

## Step 1: Connect to openEO Processing Backend
To begin, we establish a connection to the openEO backend. This connection is essential for accessing and processing EO data.

In [2]:
backend = 'cdse' 
# establish the connection to the selected backend
collection_options = {'S2_collection': 'SENTINEL2_L2A', 'S1_collection': 'SENTINEL1_GRD'}
connection = openeo.connect("openeo-staging.dataspace.copernicus.eu")
connection.authenticate_oidc()
# We call again the standard processing options for feature generation
processing_options = get_standard_processing_options(provider=backend, task='feature_generation')

Authenticated using refresh token.


## Step 2: Specify Space and Time Context
Define the spatial and temporal parameters for the data cube. This step ensures that only relevant data is loaded and processed during inference.

Important note, the job options were seme-optimised in function of the input size. Larger extents, will have higher memory requirements.

In [3]:
# the time context is given by start and end date
start = '2021-01-01'
end = '2022-01-01'   # the end is always exclusive
AOI = {
    'east': 4880000,
    'south': 2896000,
    'west': 4876000,
    'north': 2900000,
    'crs': 'EPSG:3035'
}

#note these job options were optimised for the spatiotemporal extent given above
job_options = {'driver-memory': '1000m',
 'driver-memoryOverhead': '1000m',
 'executor-memory': '1500m',
 'executor-memoryOverhead': '1500m',
 'python-memory': '4000m',
 'max-executors': 20,
 "udf-dependency-archives": [
        "https://s3.waw3-1.cloudferro.com/swift/v1/project_dependencies/onnx_dependencies_1.16.3.zip#onnx_deps"
        ]
 }

## Step 3: Define the input cube

Here we create the input datacube for the ML network inference

In [6]:
# we link towards the used model

#create the progress graph for the feature cube
# define the S1/S2 processed feature cube (Note: do not set spatial extent since we had it over in the end)
data_cube = generate_master_feature_cube(connection,
                                            AOI,
                                            start,
                                            end,
                                            **collection_options,
                                            **processing_options)

# load the DEM from a CDSE collection
DEM = connection.load_collection(
    "COPERNICUS_30",
    bands=["DEM"])
# reduce the temporal domain since copernicus_30 collection is "special" and feature only are one time stamp
DEM = DEM.reduce_dimension(dimension='t', reducer=lambda x: x.last(ignore_nodata=True))
# resample the cube to 10m and EPSG of corresponding 20x20km grid tile
DEM = DEM.resample_spatial(projection=processing_options['target_crs'],
                            resolution=processing_options['resolution'],
                            method="bilinear").filter_bbox(AOI)
# merge into the S1/S2 data cube
data_cube = data_cube.merge_cubes(DEM)

# load the WERN features from public STAC
WENR = connection.load_stac("https://catalogue.weed.apex.esa.int/collections/wenr_features")
# resample the cube to 10m and EPSG of corresponding 20x20km grid tile
WENR = WENR.resample_spatial(projection=processing_options['target_crs'],
                                resolution=processing_options['resolution'],
                                method="near").filter_bbox(AOI)
# drop the time dimension
try:
    WENR = WENR.drop_dimension('t')
except:
    # workaround if we still have the client issues with the time dimensions for STAC dataset with only one time stamp
    WENR.metadata = WENR.metadata.add_dimension("t", label=None, type="temporal")
    WENR = WENR.drop_dimension('t')

# merge into the S1/S2 data cube
data_cube = data_cube.merge_cubes(WENR)

## Step 4: Multimodel inference.

Here we create and start an openeo job which performs the inference of all onnx models in the provided folder. 

As a next step we should eveluate wheter we prefer 1 logn running job over multiple parallel openEO jobs. 

In [9]:
from datetime import datetime

filename_prefix = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
file_path = "" + filename_prefix

#we pass the model url as context information within the UDF
udf  = openeo.UDF.from_file(
        getUDFpath('udf_catboost_inference.py'),
        runtime = "Python",
        version="3.11",
        context={
            "model_id": "EUNIS2021plus_EU_v1_2024_PAN"
                }
)

# Apply the UDF to the data cube.
catboost_classification = data_cube.apply_dimension(
    process=udf, dimension = "bands")

# catboost_classification = catboost_classification.rename_labels(dimension="bands",target=output_features)
catboost_classification = catboost_classification.linear_scale_range(0,100, 0,100)


#Save each band as a seperate tiff file
save_result_options = {
        "filename_prefix": filename_prefix,
        "attach_gdalinfo_assets": True, 
    }

save_result_options["separate_asset_per_band"] = True
result_datacube = catboost_classification.save_result(
            format="GTiff",
            options = save_result_options)

# Get current datetime
file_path = "test/" 

# result_datacube = result_datacube.process(
#     "export_workspace",
#     arguments={
#         "data": result_datacube,
#         "workspace": "esa-weed-workspace",
#         "merge": file_path,  # this determines the folder structure on the s3 bucket
#     },
# )

#create and run the job
job = connection.create_job(result_datacube,
    additional=job_options,
)

job.start_and_wait()


0:00:00 Job 'j-250508120220409fb9610ed8f0f4c475': send 'start'
0:00:13 Job 'j-250508120220409fb9610ed8f0f4c475': created (progress 0%)
0:00:18 Job 'j-250508120220409fb9610ed8f0f4c475': created (progress 0%)
0:00:25 Job 'j-250508120220409fb9610ed8f0f4c475': created (progress 0%)
0:00:33 Job 'j-250508120220409fb9610ed8f0f4c475': created (progress 0%)
0:00:43 Job 'j-250508120220409fb9610ed8f0f4c475': created (progress 0%)
0:00:56 Job 'j-250508120220409fb9610ed8f0f4c475': running (progress 8.2%)
0:01:12 Job 'j-250508120220409fb9610ed8f0f4c475': running (progress 10.4%)
0:01:33 Job 'j-250508120220409fb9610ed8f0f4c475': running (progress 12.9%)
0:01:57 Job 'j-250508120220409fb9610ed8f0f4c475': running (progress 16.0%)
0:02:27 Job 'j-250508120220409fb9610ed8f0f4c475': running (progress 19.4%)
0:03:05 Job 'j-250508120220409fb9610ed8f0f4c475': running (progress 23.3%)
0:03:52 Job 'j-250508120220409fb9610ed8f0f4c475': running (progress 27.6%)
0:04:50 Job 'j-250508120220409fb9610ed8f0f4c475': run

<BatchJob job_id='j-250508120220409fb9610ed8f0f4c475'>